  # Using DataJoint for Aeon data

  ## Imports and Config

In [41]:
import datetime

import datajoint as dj
import numpy as np
import pandas as pd
import plotly.graph_objects as go

  ### DataJoint configuration

  Change the settings below to your username and password to connect to `aeon-db` on _hpc-gw1_.
  Make sure you are connected to either `hpc-gw1.hpc.swc.ucl.ac.uk` or `ssh.swc.ucl.ac.uk`

In [42]:
# dj.config["database.host"] = "aeon-db"
dj.config["database.user"] = "jburling"
# dj.config["database.password"] = "******"

 After your configuration has been set up, let DataJoint try to connect to the database.
 The output from `.list_schemas()` should list read-only schemas starting with `aeon_` in addition to your own user created schemas that have read/write access (if they exists).

In [43]:
dj.conn()
dj.list_schemas()

['u_jburling_lab',
 'u_jburling_subject',
 'u_jburling_acquisition',
 'u_jburling_tracking',
 'u_jburling_analysis',
 'u_jburling_monitor',
 'aeon_lab',
 'aeon_subject',
 'aeon_acquisition',
 'aeon_tracking',
 'aeon_analysis',
 'aeon_monitor']

  You can configure additional table display options like so:

In [44]:
dj.config["display.limit"] = 15  # limit number of displayed rows
dj.config["display.width"] = 25  # limit number of displayed columns

  Once `config` is setup, you can save your configuration to load automatically with `dj.config.save_local()`.
  This will save a _json_ file called `dj_local_conf.json` to your current working directory.
  Here's an example of the `.json` file used for Aeon:

 ```json
 {
     "database.host": "aeon-db",
     "database.password": "*********",
     "database.user": "jburling",
     "database.port": 3306,
     "database.reconnect": true,
     "connection.init_function": null,
     "connection.charset": "",
     "loglevel": "DEBUG",
     "safemode": true,
     "fetch_format": "array",
     "display.limit": 50,
     "display.width": 25,
     "display.show_tuple_count": true,
     "database.use_tls": null,
     "enable_python_native_blobs": true,
     "stores": {
         "djstore": {
             "protocol": "file",
             "location": "/nfs/nhome/live/jburling/djstore",
             "stage": "/nfs/nhome/live/jburling/djstore"
         }
     },
     "custom": {
         "database.prefix": "aeon_",
         "repository_config": {
             "ceph_aeon": "/ceph/aeon"
         }
     }
 }
 ```

 ### Virtual modules vs. import

  You have the option to create _virtual_ modules using `dj.create_virtual_module` or import the schemas directly from the `aeon.dj_pipeline` submodule.
  Virtual modules can't autopopulate the tables but also don't require the `aeon` package code to access and query the data.
  If you have access to the latest `dj_pipeline` branch on the `aeon_mecha` repo, and your environment is setup and the `aeon` package installed, you can set `_use_virtual_module=False`.

In [45]:
_use_virtual_module = True

  The main data tables on `aeon-db` are saved as schemas starting with `"aeon_"`. We'll use this prefix to create the virtual modules.

In [46]:
_db_prefix = "aeon_"

if _use_virtual_module:
    acquisition = dj.create_virtual_module("acquisition", _db_prefix + "acquisition")
    analysis = dj.create_virtual_module("analysis", _db_prefix + "analysis")
    lab = dj.create_virtual_module("lab", _db_prefix + "lab")
    subject = dj.create_virtual_module("subject", _db_prefix + "subject")
    tracking = dj.create_virtual_module("tracking", _db_prefix + "tracking")

else:
    dj.config.update(custom={**dj.config.get("custom", {}), "database.prefix": _db_prefix})
    from aeon.dj_pipeline import acquisition, analysis, lab, subject, tracking


 ## Helper functions
 
 Helper functions used later for concatenating position tracking slices.

In [47]:
def check_fetch_len(key, length=1):
    """
    Check that a key is of a certain length

    :param key: A key must be a list, query, or pandas DF (not a dict)
    :type key: list, QueryExpression, DataFrame
    :param length: Length to use to check key, defaults to 1
    :type length: int, optional
    :raises ValueError: Key is incorrect length
    """
    assert isinstance(key, (list, dj.expression.QueryExpression, pd.DataFrame))
    if not len(key) == length:
        raise ValueError(f"Key must be of length {length}")


def position_concat(session_key, acquisition, tracking, pixel_scale=0.00192):
    """
    Concatenate position data into a single pandas DataFrame

    :param session_key: a key for a single session
    :type session_key: [type]
    :param acquisition: DataJoint module
    :type acquisition: dj.Schema
    :param tracking: DataJoint module
    :type tracking: dj.Schema
    :param pixel_scale: convert pixels to mm, defaults to 0.00192
    :type pixel_scale: float, optional
    :return: A DataFrame representation of the table
    :rtype: pd.DataFrame
    """
    sess_key = (acquisition.Session() & session_key).fetch(as_dict=True)

    check_fetch_len(sess_key, 1)
    sess_key = sess_key[0]

    to_expand = [
        "timestamps",
        "position_x",
        "position_y",
        "position_z",
        "area",
        "speed",
    ]

    pos_arr = (tracking.SubjectPosition() & sess_key).fetch(
        *to_expand, order_by="time_slice_start"
    )

    for idx, field in enumerate(to_expand):
        col_data = pos_arr[idx]
        col_data = np.concatenate(col_data)
        if field != "timestamps":
            col_data *= pixel_scale
        sess_key[field] = col_data

    return pd.DataFrame(sess_key).set_index("timestamps")



  ## Relevant tables
  The following tables are important for querying data needed for plotting and general summary statistics.

 _Note:_ to view a subset of a tables' contents, call it like a function, e.g., `acquisition.Session()` and not `acquisition.Session`.

In [48]:
# basic info about a session
acquisition.Session()

experiment_name e.g exp0-a,subject,session_start
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399
exp0.1-r0,BAA-1099790,2021-06-04 07:46:35.178850
exp0.1-r0,BAA-1099790,2021-06-07 07:35:12.285250
exp0.1-r0,BAA-1099790,2021-06-07 08:16:34.970530
exp0.1-r0,BAA-1099790,2021-06-08 08:06:19.915390
exp0.1-r0,BAA-1099790,2021-06-15 13:59:20.369440
exp0.1-r0,BAA-1099790,2021-06-16 13:21:37.711780
exp0.1-r0,BAA-1099790,2021-06-17 13:07:55.917789
exp0.1-r0,BAA-1099790,2021-06-25 13:22:59.812580
exp0.1-r0,BAA-1099790,2021-06-28 12:43:44.099679


In [49]:
# similar to Session but with `session_end` and `session_duration`, if available.
acquisition.SessionEnd()

experiment_name e.g exp0-a,subject,session_start,session_end,session_duration (hour)
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 13:18:41.202020,4.68008
exp0.1-r0,BAA-1099790,2021-06-04 07:46:35.178850,2021-06-04 12:36:28.512420,4.83148
exp0.1-r0,BAA-1099790,2021-06-07 08:16:34.970530,2021-06-07 12:35:53.705279,4.32187
exp0.1-r0,BAA-1099790,2021-06-08 08:06:19.915390,2021-06-08 13:19:31.819069,5.21997
exp0.1-r0,BAA-1099790,2021-06-15 13:59:20.369440,2021-06-15 18:31:42.297060,4.53942
exp0.1-r0,BAA-1099790,2021-06-16 13:21:37.711780,2021-06-16 17:48:21.361090,4.44546
exp0.1-r0,BAA-1099790,2021-06-17 13:07:55.917789,2021-06-17 18:31:58.763519,5.40079
exp0.1-r0,BAA-1099790,2021-06-25 13:22:59.812580,2021-06-25 18:20:10.875579,4.95307
exp0.1-r0,BAA-1099790,2021-06-28 12:43:44.099679,2021-06-28 17:56:24.699299,5.21128
exp0.1-r0,BAA-1099790,2021-06-29 13:02:00.056419,2021-06-29 13:06:25.177629,0.0736448


In [50]:
# table for position tracking data
tracking.SubjectPosition()

experiment_name e.g exp0-a,subject,session_start,chunk_start datetime of the start of a given acquisition chunk,time_slice_start datetime of the start of this time slice,timestamps (datetime) timestamps of the position data,"position_x (px) animal's x-position, in the arena's coordinate frame","position_y (px) animal's y-position, in the arena's coordinate frame","position_z (px) animal's z-position, in the arena's coordinate frame",area (px^2) animal's size detected in the camera,speed (px/s) speed
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 08:00:00,2021-06-03 08:37:52.918399,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 08:00:00,2021-06-03 08:47:52.918399,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 08:00:00,2021-06-03 08:57:52.918399,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:00:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:10:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:20:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:30:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:40:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 09:00:00,2021-06-03 09:50:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 10:00:00,2021-06-03 10:00:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


 ## Basic DataJoint queries and finding a good session

 The `*` operator performs an inner join. See this link for more details: <https://docs.datajoint.org/python/v0.13/queries/07-Join.html#join-operator>

 Here we'll combine `session_end` fields with the Session table, keeping only matching records.

In [51]:
acquisition.Session * acquisition.SessionEnd

experiment_name e.g exp0-a,subject,session_start,session_end,session_duration (hour)
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 13:18:41.202020,4.68008
exp0.1-r0,BAA-1099790,2021-06-04 07:46:35.178850,2021-06-04 12:36:28.512420,4.83148
exp0.1-r0,BAA-1099790,2021-06-07 08:16:34.970530,2021-06-07 12:35:53.705279,4.32187
exp0.1-r0,BAA-1099790,2021-06-08 08:06:19.915390,2021-06-08 13:19:31.819069,5.21997
exp0.1-r0,BAA-1099790,2021-06-15 13:59:20.369440,2021-06-15 18:31:42.297060,4.53942
exp0.1-r0,BAA-1099790,2021-06-16 13:21:37.711780,2021-06-16 17:48:21.361090,4.44546
exp0.1-r0,BAA-1099790,2021-06-17 13:07:55.917789,2021-06-17 18:31:58.763519,5.40079
exp0.1-r0,BAA-1099790,2021-06-25 13:22:59.812580,2021-06-25 18:20:10.875579,4.95307
exp0.1-r0,BAA-1099790,2021-06-28 12:43:44.099679,2021-06-28 17:56:24.699299,5.21128
exp0.1-r0,BAA-1099790,2021-06-29 13:02:00.056419,2021-06-29 13:06:25.177629,0.0736448


 Here we'll further restrict the joined table to subset based on some duration condition.
 In this case, sessions which last longer than 4 hours.

In [52]:
(acquisition.Session * acquisition.SessionEnd) & "session_duration > 4"

experiment_name e.g exp0-a,subject,session_start,session_end,session_duration (hour)
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,2021-06-03 13:18:41.202020,4.68008
exp0.1-r0,BAA-1099790,2021-06-04 07:46:35.178850,2021-06-04 12:36:28.512420,4.83148
exp0.1-r0,BAA-1099790,2021-06-07 08:16:34.970530,2021-06-07 12:35:53.705279,4.32187
exp0.1-r0,BAA-1099790,2021-06-08 08:06:19.915390,2021-06-08 13:19:31.819069,5.21997
exp0.1-r0,BAA-1099790,2021-06-15 13:59:20.369440,2021-06-15 18:31:42.297060,4.53942
exp0.1-r0,BAA-1099790,2021-06-16 13:21:37.711780,2021-06-16 17:48:21.361090,4.44546
exp0.1-r0,BAA-1099790,2021-06-17 13:07:55.917789,2021-06-17 18:31:58.763519,5.40079
exp0.1-r0,BAA-1099790,2021-06-25 13:22:59.812580,2021-06-25 18:20:10.875579,4.95307
exp0.1-r0,BAA-1099790,2021-06-28 12:43:44.099679,2021-06-28 17:56:24.699299,5.21128
exp0.1-r0,BAA-1099790,2021-06-29 13:06:34.761660,2021-06-29 17:11:26.405819,4.08101


 Well define sessions with end times and long durations as `good_sessions`, and use that query table to fetch some data from the database.

In [53]:
good_sessions = (acquisition.Session * acquisition.SessionEnd) & "session_duration > 4"
subject, session_duration = good_sessions.fetch("subject", "session_duration")
session_duration[:5]  # print first five only

array([4.68008, 4.83148, 4.32187, 5.21997, 4.53942])

 If you use the special string `"KEY"`, then you can fetch only the primary keys from the table.

In [54]:
good_session_keys = good_sessions.fetch("KEY")
good_session_keys

[{'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 3, 8, 37, 52, 918399)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 4, 7, 46, 35, 178850)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 7, 8, 16, 34, 970530)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 8, 8, 6, 19, 915390)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 15, 13, 59, 20, 369440)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 16, 13, 21, 37, 711780)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session_start': datetime.datetime(2021, 6, 17, 13, 7, 55, 917789)},
 {'experiment_name': 'exp0.1-r0',
  'subject': 'BAA-1099790',
  'session

 From the list of good session primary keys, select a single session to use for further queries. Here we'll just take the first one.

In [55]:
session_key = good_session_keys[0]

 The object `session_key` is a dictionary of primary keys and their values to be used to select a valid session from different tables.

In [56]:
session_key

{'experiment_name': 'exp0.1-r0',
 'subject': 'BAA-1099790',
 'session_start': datetime.datetime(2021, 6, 3, 8, 37, 52, 918399)}

### Alternative methods to find a single session

Find a session for given a subject ID and a star time criterion.

In [57]:
session_conditional = (
    good_sessions & 'subject = "BAA-1099790"' & 'session_start > "2021-08-01"'
)

session_key = session_conditional.fetch("KEY")[0]
session_conditional

experiment_name e.g exp0-a,subject,session_start,session_end,session_duration (hour)
exp0.1-r0,BAA-1099790,2021-08-10 08:18:49.381820,2021-08-10 12:55:58.479579,4.61919


Find a session based on the one with the most time slices

1. In `SubjectPosition`, group by all the unique combinations of primary keys found in `good_sessions`, then count the number of `time_slice_start` entries for each group
2. Get the max number of time slice counts
3. Use the max stored in `max_count` to filter `time_counts` and get the last primary key found and use it for `session_key`

In [58]:
time_counts = good_sessions.aggr(tracking.SubjectPosition, n_obs="count(time_slice_start)")
max_count = time_counts.fetch("n_obs").max()
session_key = (time_counts & {"n_obs": max_count}).fetch("KEY")[-1:]
time_counts

experiment_name e.g exp0-a,subject,session_start,n_obs calculated attribute
exp0.1-r0,BAA-1099790,2021-06-03 08:37:52.918399,29
exp0.1-r0,BAA-1099790,2021-06-04 07:46:35.178850,30
exp0.1-r0,BAA-1099790,2021-06-07 08:16:34.970530,27
exp0.1-r0,BAA-1099790,2021-06-08 08:06:19.915390,32
exp0.1-r0,BAA-1099790,2021-06-15 13:59:20.369440,29
exp0.1-r0,BAA-1099790,2021-06-16 13:21:37.711780,27
exp0.1-r0,BAA-1099790,2021-06-17 13:07:55.917789,34
exp0.1-r0,BAA-1099790,2021-06-25 13:22:59.812580,31
exp0.1-r0,BAA-1099790,2021-06-28 12:43:44.099679,32
exp0.1-r0,BAA-1099790,2021-06-29 13:06:34.761660,26


 We can now use the single session as key to do restrictions/subsetting on other DataJoint tables.

 Subsetting position data for a single session:

In [59]:
tracking.SubjectPosition & session_key

experiment_name e.g exp0-a,subject,session_start,chunk_start datetime of the start of a given acquisition chunk,time_slice_start datetime of the start of this time slice,timestamps (datetime) timestamps of the position data,"position_x (px) animal's x-position, in the arena's coordinate frame","position_y (px) animal's y-position, in the arena's coordinate frame","position_z (px) animal's z-position, in the arena's coordinate frame",area (px^2) animal's size detected in the camera,speed (px/s) speed
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 11:00:00,2021-08-12 11:35:38.376349,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 11:00:00,2021-08-12 11:45:38.376349,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 11:00:00,2021-08-12 11:55:38.376349,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:00:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:10:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:20:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:30:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:40:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 12:00:00,2021-08-12 12:50:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2021-08-12 13:00:00,2021-08-12 13:00:00,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


 ### Additional tables of interest

 See the diagram from `aeon/dj_pipeline/docs/notebooks/diagram.svg` for more tables.

In [60]:
# summary statistics for one session
analysis.SessionSummary() & session_key

experiment_name e.g exp0-a,subject,session_start,total_distance_travelled (m) total distance the animal travelled during this session,total_pellet_count total pellet delivered (triggered) for all patches during this session,total_wheel_distance_travelled total wheel distance for all patches,change_in_weight weight change before/after the session
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,2226.82,136,15276.8,3.4


In [61]:
# summary statistics for food pathces for a single session
analysis.SessionSummary.FoodPatch & session_key

experiment_name e.g exp0-a,subject,session_start,food_patch_serial_number,food_patch_install_time time of the food_patch placed and started operation at this position,pellet_count number of pellets being delivered (triggered) by this patch during this session,wheel_distance_travelled wheel travel distance during this session for this patch
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,COM4,2021-06-02 23:49:41,44,6059.36
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,COM7,2021-06-02 23:49:41,92,9217.45


In [62]:
# summary statistics for time in arena or corridor for a session
analysis.SessionTimeDistribution & session_key



experiment_name e.g exp0-a,subject,session_start,time_fraction_in_corridor fraction of time the animal spent in the corridor in this session,in_corridor array of boolean for if the animal is in the corridor (same length as position data),time_fraction_in_arena fraction of time the animal spent in the arena in this session,in_arena array of boolean for if the animal is in the arena (same length as position data)
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,0.189389,=BLOB=,0.401186,=BLOB=


In [63]:
# summary statistics for time in food patch for a session
analysis.SessionTimeDistribution.FoodPatch & session_key

experiment_name e.g exp0-a,subject,session_start,food_patch_serial_number,food_patch_install_time time of the food_patch placed and started operation at this position,time_fraction_in_patch fraction of time the animal spent on this patch in this session,in_patch array of boolean for if the animal is in this patch (same length as position data)
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,COM4,2021-06-02 23:49:41,0.0522283,=BLOB=
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,COM7,2021-06-02 23:49:41,0.0826749,=BLOB=


In [64]:
# summary statistics for time in nest for a session
analysis.SessionTimeDistribution.Nest & session_key

experiment_name e.g exp0-a,subject,session_start,arena_name unique name of the arena (e.g. circular_2m),"nest nest number - e.g. 1, 2, ...",time_fraction_in_nest fraction of time the animal spent in this nest in this session,in_nest array of boolean for if the animal is in this nest (same length as position data)
exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,circle-2m,1,0.29403,=BLOB=


 Using these tables together in practice

In [65]:
# join Session timestamp tables then subset using `session_key`
session_times = (acquisition.Session * acquisition.SessionEnd) & session_key

# extract session time stamps from the above query table
start_time, end_time = session_times.fetch1("session_start", "session_end")

# string to further restrict patch events to a specific time range
time_range_str = f"event_time BETWEEN '{start_time}' AND '{end_time}'"
patch_events = acquisition.FoodPatchEvent & time_range_str
patch_events

experiment_name e.g exp0-a,chunk_start datetime of the start of a given acquisition chunk,food_patch_serial_number,food_patch_install_time time of the food_patch placed and started operation at this position,event_number,event_time event time,event_code
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,2,2021-08-12 11:37:34.841504,35
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,3,2021-08-12 11:37:35.531616,32
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,4,2021-08-12 11:37:46.893504,35
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,5,2021-08-12 11:37:47.509119,32
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,6,2021-08-12 11:37:56.221504,35
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,7,2021-08-12 11:37:56.844287,32
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,8,2021-08-12 11:38:05.269504,35
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,9,2021-08-12 11:38:05.954175,32
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,10,2021-08-12 11:38:17.021503,35
exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,11,2021-08-12 11:38:17.731904,32


In [66]:
# lookup table mapping codes to event types
acquisition.EventType()

event_code,event_type
32,PelletDetected
35,TriggerPellet
1000,No Events


In [67]:
# joint `patch_events` data with EventType to combine with event types and event codes
acquisition.EventType * patch_events

event_code,experiment_name e.g exp0-a,chunk_start datetime of the start of a given acquisition chunk,food_patch_serial_number,food_patch_install_time time of the food_patch placed and started operation at this position,event_number,event_type,event_time event time
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,3,PelletDetected,2021-08-12 11:37:35.531616
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,5,PelletDetected,2021-08-12 11:37:47.509119
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,7,PelletDetected,2021-08-12 11:37:56.844287
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,9,PelletDetected,2021-08-12 11:38:05.954175
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,11,PelletDetected,2021-08-12 11:38:17.731904
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,13,PelletDetected,2021-08-12 11:38:30.181215
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,15,PelletDetected,2021-08-12 11:38:40.250624
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,17,PelletDetected,2021-08-12 11:48:42.988480
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,19,PelletDetected,2021-08-12 11:48:57.383520
32,exp0.1-r0,2021-08-12 11:00:00,COM4,2021-06-02 23:49:41,21,PelletDetected,2021-08-12 11:49:08.389855


  ## Generating the traceplot

  Use the `session_key` to subset data from `SubjectPosition` and concatenating all individual time slices into a single data frame.
  A similar concatenation function is also found in the `aeon` package under `aeon.dj_pipeline.tracking` and called `SubjectPosition.get_session_position()`

In [68]:
pos_data = position_concat(session_key, acquisition, tracking, pixel_scale=1)
pos_data = pos_data[:50_000]  # subsetting for performance reasons
pos_data

,experiment_name,subject,session_start,position_x,position_y,position_z,area,speed
timestamps,,,,,,,,
2021-08-12 11:35:38.388352,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,231.772873,499.458740,0.0,260.5,31.286059
2021-08-12 11:35:38.408384,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,231.190659,499.226776,0.0,267.5,31.286059
2021-08-12 11:35:38.428352,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,230.199493,499.026276,0.0,266.5,50.643089
2021-08-12 11:35:38.448384,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,229.129120,498.972076,0.0,268.5,53.501643
2021-08-12 11:35:38.468352,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,228.031189,498.720490,0.0,272.5,56.409617
...,...,...,...,...,...,...,...,...
2021-08-12 11:52:18.296416,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,1260.555054,592.794128,0.0,503.5,78.309202
2021-08-12 11:52:18.316384,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,1260.686035,593.698547,0.0,491.5,45.765941
2021-08-12 11:52:18.336416,exp0.1-r0,BAA-1099793,2021-08-12 11:35:38.376349,1260.746582,594.414978,0.0,486.0,35.891800


  ### Additional plotting helpers taken from `joacorapela`

In [69]:
def get_trajectory_trace(x, y, time_stamps):
    trace = go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker={
            "color": time_stamps,
            "colorscale": "Rainbow",
            "colorbar": {"title": "Time"},
        },
        customdata=time_stamps,
        hovertemplate="<b>x:</b>%{x:.3f}<br><b>y</b>:%{y:.3f}<br><b>time</b>:%{customdata} sec",
        showlegend=False,
    )
    return trace


def get_patches_traces(patches_coordinates, fill_color="gray", fill_opacity=0.5):
    patches_traces = []
    for i in range(patches_coordinates.shape[0]):
        patch_lower_x, patch_higher_x, patch_lower_y, patch_higher_y = patches_coordinates.loc[
            i, ["lower_x", "higher_x", "lower_y", "higher_y"]
        ]
        patch_xs = [
            patch_lower_x,
            patch_higher_x,
            patch_higher_x,
            patch_lower_x,
            patch_lower_x,
        ]
        patch_ys = [
            patch_lower_y,
            patch_lower_y,
            patch_higher_y,
            patch_higher_y,
            patch_lower_y,
        ]
        patch_trace = go.Scatter(
            x=patch_xs,
            y=patch_ys,
            fill="toself",
            fillcolor=fill_color,
            opacity=fill_opacity,
            mode="none",
            showlegend=False,
        )
        patches_traces.append(patch_trace)
    return patches_traces

In [87]:
x = pos_data["position_x"].to_numpy()
y = pos_data["position_y"].to_numpy()
session_start = pos_data.session_start[0].to_numpy()
time_stamps = (pos_data.index - session_start).total_seconds().to_numpy()
duration_sec = (acquisition.SessionEnd & session_key).fetch1("session_duration") * 3600

title = "Start {:s}, (max={:.02f} sec)".format(str(session_start), duration_sec)

patches_coordinates = pd.DataFrame(
    data=np.matrix("584,597,815,834;614,634,252,271"),
    columns=["lower_x", "higher_x", "lower_y", "higher_y"],
)

nest_coordinates = pd.DataFrame(
    data=np.matrix("170,260,450,540"), columns=["lower_x", "higher_x", "lower_y", "higher_y"]
)

fig = go.Figure()

trace = get_trajectory_trace(x, y, time_stamps)

fig.add_trace(trace)

patches_traces = get_patches_traces(patches_coordinates=patches_coordinates)

for patch_trace in patches_traces:
    fig.add_trace(patch_trace)

nest_trace = get_patches_traces(patches_coordinates=nest_coordinates)[0]

fig.add_trace(nest_trace)

fig.update_layout(
    title=title,
    xaxis_title="x (pixels)",
    yaxis_title="y (pixels)",
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    width=700,
    height=700,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
)


  # Appendix
  ## `aeon` package setup and requirements
  
  TODO

 ## Additional helper functions

 The functions below help to find complete sessions using information stored among different tables.

In [ ]:
def join_timestamps(acquisition, keep_null=False):
    # get subject keys, session start & end, time_slice start & end
    return acquisition.Session.join(acquisition.SessionEnd, left=keep_null).join(
        acquisition.TimeSlice, left=keep_null
    )


def keys_complete_sessions(acquisition):
    all_timestamps = join_timestamps(acquisition, keep_null=True)

    # session_start time but no chunk_start time
    nonstart_sessions = acquisition.Session & (all_timestamps & "ISNULL(chunk_start)")

    # session_start time but no session_end time
    ongoing_sessions = acquisition.Session & (all_timestamps & "ISNULL(session_end)")

    return acquisition.Session - nonstart_sessions - ongoing_sessions


def keys_complete_tracking(acquisition, tracking):
    sessions = keys_complete_sessions(acquisition)
    tracking_data = sessions.join(tracking.SubjectPosition, left=True)

    incomplete_tracking_sessions = acquisition.Session & (tracking_data & "ISNULL(timestamps)")

    return sessions - incomplete_tracking_sessions